In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import keras

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')

In [0]:
train,test = train_test_split(df)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}


In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [10]:
MAX_NB_WORDS = 150000

texts_train =X_train.astype(str)
texts_test = X_test.astype(str)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 438172 unique tokens.


In [0]:
from keras.preprocessing.sequence import pad_sequences

In [12]:
MAX_SEQUENCE_LENGTH = 200

x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)


Shape of data tensor: (710726, 200)
Shape of data test tensor: (236909, 200)


In [13]:


Y_train = to_categorical(np.asarray(Y_train))
Y_test = to_categorical(np.asarray(Y_test))
print('Shape of label tensor:', Y_train.shape)
print('Shape of label tensor:', Y_test.shape)


Shape of label tensor: (710726, 87)
Shape of label tensor: (236909, 87)


In [0]:
from keras import backend as K
import tensorflow as tf
import dill

In [0]:
def categorical_focal_loss(gamma=2., alpha=.25):

  

  def categorical_focal_loss_fixed(y_true, y_pred):

        

    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

    epsilon = K.epsilon()

    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

    cross_entropy = -y_true * K.log(y_pred)

    loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

    return K.sum(loss, axis=1)

      

  return categorical_focal_loss_fixed


  

def categorical_focal_loss_fixed(y_true, y_pred):

        

  y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

  epsilon = K.epsilon()
  y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

  cross_entropy = -y_true * K.log(y_pred)

  loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

  return K.sum(loss, axis=1)

In [24]:
from keras.layers import Dense, Input, Flatten,GlobalMaxPooling1D
from keras.layers import GlobalAveragePooling1D, Embedding , Reshape,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
VALIDATION_SPLIT = 0.20
EMBEDDING_DIM = 300 
N_CLASSES = 87
MAX_SEQUENCE_LENGTH = 200

filter_sizes = [2,3,4,5]
num_filters = 512
embedding_dim = 300
drop = 0.5


VALIDATION_SPLIT = 0.20
EMBEDDING_DIM = 300 
N_CLASSES = 87
MAX_SEQUENCE_LENGTH = 200

filter_sizes = [2,3,4,5,6,7]
num_filters = 512
embedding_dim = 300
drop = 0.5
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)
print(embedded_sequences.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedded_sequences)
print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)
maxpool_3 = MaxPooling2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[3] + 1, 1), strides=(1,1), padding='valid')(conv_3)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2,maxpool_3])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)


l_dense = Dense(309, activation='relu')(dropout)
l_dense_n = Dense(150, activation='relu')(l_dense)

# this creates a model that includes
output = Dense(units=87, activation='softmax')(l_dense_n)

# this creates a model that includes
model = Model(inputs=sequence_input, outputs=output)

check = ModelCheckpoint('weights_cnn_dude_please_fl.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=['accuracy'])
model.summary()

(?, 200, 300)
(?, 200, 300, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 200, 300)     45000000    input_3[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 200, 300, 1)  0           embedding_3[0][0]                
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 199, 1, 512)  307712      reshape_3[0][0]                  
______________________________________________________________________________

In [23]:
model = keras.models.load_model('weights_cnn_dude_please_fl.hdf5',custom_objects={'categorical_focal_loss':categorical_focal_loss})
model.summary()

ValueError: ignored

In [27]:
model.fit(x_train, Y_train, validation_data=(x_test,Y_test),
          nb_epoch=20, batch_size=15,callbacks=[check])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 710726 samples, validate on 236909 samples
Epoch 1/20
710726/710726 [==============================] - 5463s 8ms/step - loss: 0.0440 - acc: 0.9142 - val_loss: 0.0503 - val_acc: 0.9066

Epoch 00001: val_acc improved from 0.89593 to 0.90662, saving model to weights_cnn_dude_please_fl.hdf5
Epoch 2/20
710726/710726 [==============================] - 5462s 8ms/step - loss: 0.0335 - acc: 0.9272 - val_loss: 0.0491 - val_acc: 0.9073

Epoch 00002: val_acc improved from 0.90662 to 0.90730, saving model to weights_cnn_dude_please_fl.hdf5
Epoch 3/20
710726/710726 [==============================] - 5460s 8ms/step - loss: 0.0269 - acc: 0.9364 - val_loss: 0.0489 - val_acc: 0.9088

Epoch 00003: val_acc improved from 0.90730 to 0.90877, saving model to weights_cnn_dude_please_fl.hdf5
Epoch 4/20
710726/710726 [==============================] - 5435s 8ms/step - loss: 0.0221 - acc: 0.9434 - val_loss: 0.0514 - val_acc: 0.9078

Epoch 00004: val_acc did not improve from 0.90877
Epoch 5/20
 16380/710

KeyboardInterrupt: ignored

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(x_test)


In [35]:
from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced
#Y_test = np.argmax(Y_test,axis=1)
#output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test,digits=5))

              precision    recall  f1-score   support

           0    0.97698   0.98771   0.98231     29857
           1    0.97407   0.97269   0.97338      5639
           2    0.82917   0.80453   0.81667      1279
           3    0.94055   0.93698   0.93876      1317
           4    0.79361   0.57903   0.66955      1202
           5    0.95540   0.96232   0.95884      2760
           6    0.93930   0.93037   0.93481      4208
           7    0.94472   0.95185   0.94827      2783
           8    0.91282   0.92128   0.91703      7660
           9    0.93832   0.92718   0.93272     13897
          10    0.96896   0.98188   0.97537     15449
          11    0.56522   0.36449   0.44318       107
          12    0.76371   0.84363   0.80169      7329
          13    0.92947   0.96470   0.94676     13484
          14    0.94793   0.95886   0.95337      8411
          15    0.87568   0.82781   0.85107      2323
          16    0.84374   0.88795   0.86528      2481
          17    0.97621   0

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [40]:
MAX_SEQUENCE_LENGTH = 200

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

Shape of data test tensor: (200000, 200)


In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [45]:
pred.shape


(200000,)

In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [50]:
for index, row tst.iterrow():
  if 'عمده' in row['desc'] | 'عمده '

,Unnamed: 0,archive_by_user,city,created_at,desc,id,image_count,platform,price,title,cat1_2_3,cat1,cat2_3,cat2,cat3
39,39,False,Tehran,Tuesday 05AM,عرضه رب در دبه های 5 کیلویی و 10 کیلویی با کیف...,13807426594969,1,mobile,4000,فروش رب 100% طبیعی به قیمت مناسب,businesses|batch|nan,businesses,batch|nan,batch,nan
403,403,False,Karaj,Tuesday 10AM,فروش عمده لباسهای بچگانه و نوزادی موجود در مغا...,67249759381297,4,web,-1,فروش عمده پوشاک نوزادی و بچه گانه,businesses|batch|nan,businesses,batch|nan,batch,nan
407,407,False,Tehran,Tuesday 08PM,ذرت منجمد هندی به مقدار 200 تن موجودی هست با ق...,46874220201610,0,mobile,6500,ذرت منجمد هندی,businesses|batch|nan,businesses,batch|nan,batch,nan
413,413,True,Kermanshah,Tuesday 08AM,فروش جزیی و کلی الو بخارا و شیره انگور کاملا ا...,6203541126822,1,mobile,18000,الو بخارا(خورشتی)و شیره,businesses|batch|nan,businesses,batch|nan,batch,nan
543,543,False,Shiraz,Friday 11PM,نوار تیپ آبیاری قطره ای\r\r\r\nقیمت درب کارخان...,14933362782668,0,mobile,-1,نوار تیپ آبیاری قطره ای,businesses|batch|nan,businesses,batch|nan,batch,nan
587,587,True,Tehran,Saturday 10PM,فروش الو بخارا و برگ هلودر بسته بندی یک کیلویی...,64844556304096,2,mobile,21000,الو بخارا,businesses|batch|nan,businesses,batch|nan,batch,nan
618,618,False,Karaj,Monday 01PM,فروش عمده و تکی قاب فرش با انواع رنگ و طرح مخت...,27267540303611,5,mobile,60000,فروش عمده و تکی قاب فرش,businesses|batch|nan,businesses,batch|nan,batch,nan
665,665,False,Tehran,Wednesday 04PM,دفاتر کودک فانتزی اکلیلی80برگ هستند.هر دفتر 2 ...,6663642225570,1,web,-1,لوازم تحریر,businesses|batch|nan,businesses,batch|nan,batch,nan
983,983,True,Tehran,Tuesday 06PM,انواع سفال هفت سین خام و رنگ شده به صورت عمده ...,1789215678566,2,mobile,-1,سفال هفت سین به صورت عمده خام و رنگی,businesses|batch|nan,businesses,batch|nan,batch,nan
1134,1134,False,Isfahan,Saturday 04PM,باسلام\r\r\r\nتهیه سوسیس و کالباس خانگی.\r\r\r...,59163052133631,2,mobile,45000,تهیه سوسی و کالباس خانگی,businesses|batch|nan,businesses,batch|nan,batch,nan


In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [53]:
tst

,cat1,cat2,cat3
0,for-the-home,furniture-and-home-decore,tables-and-chairs
1,electronic-devices,mobile-tablet,tablet
2,for-the-home,furniture-and-home-decore,tv-and-stereo-furniture
3,businesses,equipments-and-machinery,industrial
4,vehicles,cars,light
5,leisure-hobbies,animals,birds
6,personal,health-beauty,
7,electronic-devices,game-consoles-and-video-games,
8,for-the-home,furniture-and-home-decore,antiques-and-art
9,vehicles,cars,light


In [0]:
tst.to_csv("tst_deep_2004.csv")